In [1]:
from utils import (
    get_cross_validation_results,
    get_final_model_performance,
    TransactionDataset,
    TuneHyperParams,
)

from constants import Columns, ModelConstants

In [3]:
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import (
    OneHotEncoder,
    OrdinalEncoder,
    FunctionTransformer,
    Normalizer,
    PowerTransformer,
    RobustScaler
)
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_transformer, make_column_selector


In [4]:
data = TransactionDataset().get_training_test_split()

In [5]:
log_transform = FunctionTransformer(
    func=np.log1p, inverse_func=np.expm1, check_inverse=False
)

order_mapping = [['0', '0.2', '0.4', '0.6', '0.8', '1']]

norm_and_one_hot_transforms = make_column_transformer(
    (OneHotEncoder(), [Columns.CUSTOMER_TYPE]),
    (OrdinalEncoder(categories=order_mapping), [Columns.SPECIFIC_HOLIDAY]),
    (Normalizer(), make_column_selector(dtype_include="number"))
)

log_and_ordinal_transforms = make_column_transformer(
    (OneHotEncoder(), [Columns.CUSTOMER_TYPE]),
    (OrdinalEncoder(categories=order_mapping), [Columns.SPECIFIC_HOLIDAY]),
    (log_transform, make_column_selector(dtype_include="number"))
)

power_transforms = make_column_transformer(
    (OneHotEncoder(), [Columns.CUSTOMER_TYPE]),
    (OrdinalEncoder(categories=order_mapping), [Columns.SPECIFIC_HOLIDAY]),
    (PowerTransformer(), make_column_selector(dtype_include="number"))
)

robust_transformation = make_column_transformer(
    (OneHotEncoder(), [Columns.CUSTOMER_TYPE]),
    (OrdinalEncoder(categories=order_mapping), [Columns.SPECIFIC_HOLIDAY]),
    (RobustScaler(), make_column_selector(dtype_include="number"))
)

In [7]:
knn_classifier = KNeighborsClassifier(
    n_jobs=ModelConstants.N_JOBS
)

In [8]:
column_transforms_only_model = Pipeline(
    [
        ("column_transformation", log_and_ordinal_transforms),
        ("knn", knn_classifier)
    ]
)

In [9]:
tuning_params = {
    "column_transformation": [
        log_and_ordinal_transforms,
        norm_and_one_hot_transforms,
        power_transforms,
        robust_transformation,
        "passthrough"
    ],

    "knn__n_neighbors": [5, 7, 9, 11, 13, 15, 17, 19, 21],
    "knn__weights": ["uniform", "distance"],
}

In [10]:
best_model = (
    TuneHyperParams()
    .random_grid_search(
     column_transforms_only_model,
     tuning_params
    )
    .fit_model(
        data.TRAINING.predictors,
        data.TRAINING.outcome
    )
    .get_best_model()
)

{'knn__weights': 'uniform', 'knn__n_neighbors': 9, 'column_transformation': ColumnTransformer(transformers=[('onehotencoder', OneHotEncoder(),
                                 ['Customer_Type']),
                                ('ordinalencoder',
                                 OrdinalEncoder(categories=[['0', '0.2', '0.4',
                                                             '0.6', '0.8',
                                                             '1']]),
                                 ['SpecificHoliday']),
                                ('functiontransformer',
                                 FunctionTransformer(check_inverse=False,
                                                     func=<ufunc 'log1p'>,
                                                     inverse_func=<ufunc 'expm1'>),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x139079570>)])}
Best parameter (CV score: 0.593):


In [11]:
get_cross_validation_results(best_model, data.TRAINING.predictors, data.TRAINING.outcome)

,Test,Score
0,fit_time,0.019502
1,score_time,0.069739
2,test_F1 Score,0.608730
3,test_Accuracy,0.890415
4,test_Balanced Accuracy,0.750307
5,test_Matthew's Correlation Coefficient,0.551942


In [12]:
get_final_model_performance(best_model, data.TRAINING, data.TESTING)

,Test,Score
0,F1 Score,0.610370
1,Accuracy,0.892257
2,Balanced Accuracy,0.810567
3,Matthew's Correlation Coefficient,0.555619
